# BAM Production Workflows (v1.4.0)

**Duration**: 45-60 minutes  
**Level**: Advanced  
**Prerequisites**: Basic BAM knowledge (see notebook 05)

---

## What's New in v1.4.0

This notebook demonstrates **production-ready workflows** using v1.4.0 features:

### Extended Tag Parsing:
- `record.get_int(tag)` - Direct integer tag access
- `record.get_string(tag)` - Direct string tag access
- `record.edit_distance()` - NM tag convenience (mismatches)
- `record.alignment_score()` - AS tag convenience
- `record.read_group()` - RG tag convenience
- `record.md_string()` - MD tag convenience (mismatch details)

### Statistics Functions:
- `biometal.insert_size_distribution()` - Paired-end QC
- `biometal.edit_distance_stats()` - Alignment quality assessment
- `biometal.strand_bias()` - Variant calling QC
- `biometal.alignment_length_distribution()` - RNA-seq QC

---

## Five Production Workflows

1. **Quality Control Pipeline** - Comprehensive alignment QC
2. **Paired-End Insert Size Analysis** - Library preparation QC
3. **Variant Calling Preparation** - Coverage and strand bias
4. **RNA-seq Alignment QC** - Splice junction analysis
5. **Multi-Sample Tag-Based Filtering** - Read group processing

## Installation

Install biometal v1.4.0+:

```bash
pip install biometal-rs>=1.4.0
```

In [ ]:
# Import biometal and utilities
import biometal
from collections import defaultdict, Counter
import statistics
import os

# Check version
print(f"biometal version: {biometal.__version__}")
print(f"Required: 1.4.0+ for production workflows\n")

# Verify new v1.4.0 features
features = [
    'insert_size_distribution',
    'edit_distance_stats',
    'strand_bias',
    'alignment_length_distribution'
]

print("v1.4.0 Features Available:")
for feature in features:
    available = hasattr(biometal, feature)
    status = "✅" if available else "❌"
    print(f"  {status} {feature}")

## Demo Data

For this tutorial, we'll use synthetic BAM data with realistic characteristics:
- Paired-end reads (insert size ~300-500bp)
- Multiple references (chromosomes)
- Various CIGAR operations (indels, introns, clipping)
- Optional tags (NM, AS, RG, MD)

In [ ]:
# Check for test data
test_bam = "../experiments/native-bam-implementation/test-data/synthetic_100000.bam"

if os.path.exists(test_bam):
    bam_path = test_bam
    print(f"✅ Using test data: {bam_path}")
    print(f"   File size: {os.path.getsize(bam_path) / 1024:.1f} KB")
else:
    bam_path = "your_alignments.bam"  # Replace with your BAM file
    print(f"⚠️  Test data not found. Please provide your own BAM file.")
    print(f"   Set: bam_path = 'path/to/your/file.bam'")

---

# Workflow 1: Comprehensive Quality Control Pipeline

**Goal**: Generate a complete QC report for alignment quality assessment.

**Metrics**:
- Mapping statistics (rate, MAPQ distribution)
- Edit distance distribution (mismatches per read)
- Alignment score distribution
- CIGAR operation frequencies
- Strand balance
- Primary vs secondary alignment rates

**Use Case**: Pre-processing QC before downstream analysis (variant calling, quantification)

In [ ]:
def comprehensive_qc_report(bam_path: str, limit: int = 10000) -> dict:
    """
    Generate comprehensive QC report using v1.4.0 features.
    
    Args:
        bam_path: Path to BAM file
        limit: Maximum records to process (None for all)
    
    Returns:
        dict with QC metrics
    """
    reader = biometal.BamReader.from_path(bam_path)
    
    # Initialize metrics
    metrics = {
        'total': 0,
        'mapped': 0,
        'unmapped': 0,
        'primary': 0,
        'secondary': 0,
        'forward': 0,
        'reverse': 0,
        'paired': 0,
        'mapq_dist': Counter(),
        'edit_distances': [],
        'alignment_scores': [],
        'cigar_ops': Counter(),
        'read_groups': Counter(),
    }
    
    print("🔬 Running comprehensive QC analysis...\n")
    
    for record in reader:
        metrics['total'] += 1
        
        # Basic flags
        if record.is_mapped:
            metrics['mapped'] += 1
        else:
            metrics['unmapped'] += 1
        
        if record.is_primary:
            metrics['primary'] += 1
        else:
            metrics['secondary'] += 1
        
        if record.is_reverse:
            metrics['reverse'] += 1
        else:
            metrics['forward'] += 1
        
        if record.is_paired:
            metrics['paired'] += 1
        
        # MAPQ distribution
        if record.mapq is not None:
            metrics['mapq_dist'][record.mapq] += 1
        
        # NEW v1.4.0: Tag convenience methods
        edit_dist = record.edit_distance()
        if edit_dist is not None:
            metrics['edit_distances'].append(edit_dist)
        
        align_score = record.alignment_score()
        if align_score is not None:
            metrics['alignment_scores'].append(align_score)
        
        read_group = record.read_group()
        if read_group is not None:
            metrics['read_groups'][read_group] += 1
        
        # CIGAR operations
        for op in record.cigar:
            metrics['cigar_ops'][op.op_char] += op.length
        
        if limit and metrics['total'] >= limit:
            break
    
    return metrics


# Run QC analysis
qc_metrics = comprehensive_qc_report(bam_path, limit=10000)

# Generate report
total = qc_metrics['total']
print(f"📊 COMPREHENSIVE QC REPORT")
print(f"={'=' * 70}\n")
print(f"File: {bam_path}")
print(f"Records analyzed: {total:,}\n")

# Mapping statistics
print(f"MAPPING STATISTICS:")
print(f"  Mapped: {qc_metrics['mapped']:,} ({100*qc_metrics['mapped']/total:.1f}%)")
print(f"  Unmapped: {qc_metrics['unmapped']:,} ({100*qc_metrics['unmapped']/total:.1f}%)")
print(f"  Primary: {qc_metrics['primary']:,} ({100*qc_metrics['primary']/total:.1f}%)")
print(f"  Secondary/supplementary: {qc_metrics['secondary']:,} ({100*qc_metrics['secondary']/total:.1f}%)")

# Strand balance
print(f"\nSTRAND BALANCE:")
print(f"  Forward: {qc_metrics['forward']:,} ({100*qc_metrics['forward']/total:.1f}%)")
print(f"  Reverse: {qc_metrics['reverse']:,} ({100*qc_metrics['reverse']/total:.1f}%)")
strand_ratio = qc_metrics['forward'] / qc_metrics['reverse'] if qc_metrics['reverse'] > 0 else float('inf')
print(f"  Ratio: {strand_ratio:.2f}:1 {'✅ PASS' if 0.9 <= strand_ratio <= 1.1 else '⚠️  WARN'}")

# MAPQ distribution (top 5)
print(f"\nMAPQ DISTRIBUTION (top 5):")
for mapq, count in qc_metrics['mapq_dist'].most_common(5):
    print(f"  MAPQ {mapq}: {count:,} ({100*count/total:.1f}%)")

# Edit distance statistics (NEW v1.4.0)
print(f"\nEDIT DISTANCE STATISTICS (NM tag):")
if qc_metrics['edit_distances']:
    ed_mean = statistics.mean(qc_metrics['edit_distances'])
    ed_median = statistics.median(qc_metrics['edit_distances'])
    ed_stdev = statistics.stdev(qc_metrics['edit_distances']) if len(qc_metrics['edit_distances']) > 1 else 0
    print(f"  Reads with NM tag: {len(qc_metrics['edit_distances']):,}")
    print(f"  Mean: {ed_mean:.2f} mismatches/read")
    print(f"  Median: {ed_median}")
    print(f"  Std dev: {ed_stdev:.2f}")
    print(f"  Min: {min(qc_metrics['edit_distances'])}")
    print(f"  Max: {max(qc_metrics['edit_distances'])}")
else:
    print(f"  No NM tags found")

# Alignment score statistics (NEW v1.4.0)
print(f"\nALIGNMENT SCORE STATISTICS (AS tag):")
if qc_metrics['alignment_scores']:
    as_mean = statistics.mean(qc_metrics['alignment_scores'])
    as_median = statistics.median(qc_metrics['alignment_scores'])
    print(f"  Reads with AS tag: {len(qc_metrics['alignment_scores']):,}")
    print(f"  Mean: {as_mean:.2f}")
    print(f"  Median: {as_median}")
    print(f"  Min: {min(qc_metrics['alignment_scores'])}")
    print(f"  Max: {max(qc_metrics['alignment_scores'])}")
else:
    print(f"  No AS tags found")

# CIGAR operations
print(f"\nCIGAR OPERATIONS:")
total_cigar = sum(qc_metrics['cigar_ops'].values())
for op, count in sorted(qc_metrics['cigar_ops'].items()):
    print(f"  {op}: {count:,} bases ({100*count/total_cigar:.1f}%)")

# Read groups (NEW v1.4.0)
print(f"\nREAD GROUPS (RG tag):")
if qc_metrics['read_groups']:
    for rg, count in qc_metrics['read_groups'].most_common():
        print(f"  {rg}: {count:,} reads ({100*count/total:.1f}%)")
else:
    print(f"  No read groups found")

print(f"\n{'=' * 70}")
print(f"✅ QC Complete - Constant ~5 MB memory used")

---

# Workflow 2: Paired-End Insert Size Analysis

**Goal**: Analyze insert size distribution for library preparation QC.

**Metrics**:
- Insert size distribution (fragment length)
- Mean, median, mode insert size
- Standard deviation (library complexity)
- Outlier detection (adapter dimers, long inserts)

**Use Case**: 
- Quality control for paired-end library prep
- Detect adapter contamination (short inserts)
- Validate expected insert size (300-500bp typical)

**NEW v1.4.0**: `biometal.insert_size_distribution()` - Fast, built-in function

In [ ]:
def analyze_insert_sizes(bam_path: str, reference_id: int = None):
    """
    Comprehensive insert size analysis using v1.4.0 built-in function.
    
    Args:
        bam_path: Path to BAM file
        reference_id: Optional reference to analyze (None = all)
    """
    print("🔬 Analyzing insert size distribution...\n")
    
    # NEW v1.4.0: Built-in insert size distribution
    dist = biometal.insert_size_distribution(bam_path, reference_id=reference_id)
    
    if not dist:
        print("❌ No properly paired reads found")
        return
    
    # Calculate statistics
    insert_sizes = []
    for size, count in dist.items():
        insert_sizes.extend([size] * count)
    
    total_pairs = len(insert_sizes)
    mean_size = statistics.mean(insert_sizes)
    median_size = statistics.median(insert_sizes)
    mode_size = max(dist.items(), key=lambda x: x[1])[0]
    stdev_size = statistics.stdev(insert_sizes)
    
    # Percentiles
    sorted_sizes = sorted(insert_sizes)
    p5 = sorted_sizes[int(0.05 * len(sorted_sizes))]
    p95 = sorted_sizes[int(0.95 * len(sorted_sizes))]
    p99 = sorted_sizes[int(0.99 * len(sorted_sizes))]
    
    # Outliers
    short_inserts = sum(1 for s in insert_sizes if s < 100)  # Adapter dimers
    long_inserts = sum(1 for s in insert_sizes if s > 1000)  # Long fragments
    
    # Generate report
    print(f"📊 INSERT SIZE ANALYSIS")
    print(f"={'=' * 70}\n")
    print(f"File: {bam_path}")
    if reference_id is not None:
        print(f"Reference: {reference_id}")
    print(f"Properly paired reads: {total_pairs:,}\n")
    
    print(f"SUMMARY STATISTICS:")
    print(f"  Mean: {mean_size:.1f} bp")
    print(f"  Median: {median_size} bp")
    print(f"  Mode: {mode_size} bp (most common)")
    print(f"  Std dev: {stdev_size:.1f} bp")
    print(f"  Min: {min(insert_sizes)} bp")
    print(f"  Max: {max(insert_sizes)} bp")
    
    print(f"\nPERCENTILES:")
    print(f"  5th: {p5} bp")
    print(f"  95th: {p95} bp")
    print(f"  99th: {p99} bp")
    
    print(f"\nOUTLIER DETECTION:")
    short_pct = 100 * short_inserts / total_pairs
    long_pct = 100 * long_inserts / total_pairs
    print(f"  Short inserts (<100bp): {short_inserts:,} ({short_pct:.2f}%)")
    if short_pct > 5:
        print(f"    ⚠️  WARNING: High adapter dimer contamination")
    else:
        print(f"    ✅ PASS")
    
    print(f"  Long inserts (>1000bp): {long_inserts:,} ({long_pct:.2f}%)")
    if long_pct > 5:
        print(f"    ⚠️  WARNING: Unusual long fragments")
    else:
        print(f"    ✅ PASS")
    
    print(f"\nQUALITY ASSESSMENT:")
    if 300 <= mean_size <= 500:
        print(f"  ✅ Mean insert size in expected range (300-500bp)")
    else:
        print(f"  ⚠️  Mean insert size outside typical range")
    
    if stdev_size < 100:
        print(f"  ✅ Low variability (good library complexity)")
    elif stdev_size < 200:
        print(f"  ⚠️  Moderate variability")
    else:
        print(f"  ⚠️  High variability (check library prep)")
    
    # Distribution histogram (text-based)
    print(f"\nDISTRIBUTION (top 10 sizes):")
    for size, count in sorted(dist.items(), key=lambda x: -x[1])[:10]:
        pct = 100 * count / total_pairs
        bar = '█' * int(pct * 2)  # Scale bar
        print(f"  {size:4d}bp: {bar} {count:,} ({pct:.1f}%)")
    
    print(f"\n{'=' * 70}")
    print(f"✅ Analysis complete")


# Run insert size analysis
analyze_insert_sizes(bam_path)

---

# Workflow 3: Variant Calling Preparation

**Goal**: Prepare alignment metrics for variant calling QC.

**Metrics**:
- Per-position coverage
- Strand bias at positions (forward vs reverse)
- Edit distance statistics (alignment quality)
- MAPQ distribution

**Use Case**:
- Quality control before variant calling
- Identify low-coverage regions
- Detect strand bias artifacts
- Filter poor alignments

**NEW v1.4.0**: 
- `biometal.strand_bias()` - Built-in strand bias calculation
- `biometal.edit_distance_stats()` - Alignment quality metrics

In [ ]:
def variant_calling_qc(
    bam_path: str,
    reference_id: int = 0,
    positions: list = None,
    window_size: int = 100
):
    """
    Prepare variant calling QC metrics using v1.4.0 features.
    
    Args:
        bam_path: Path to BAM file
        reference_id: Reference sequence to analyze
        positions: List of positions to check (None = auto-detect)
        window_size: Window size for strand bias calculation
    """
    print("🔬 Running variant calling QC...\n")
    
    # NEW v1.4.0: Edit distance statistics
    print("📊 ALIGNMENT QUALITY (Edit Distance):")
    print(f"{'=' * 70}\n")
    
    ed_stats = biometal.edit_distance_stats(bam_path, reference_id=reference_id)
    
    print(f"  Total records: {ed_stats['total_records']:,}")
    print(f"  With NM tag: {ed_stats['with_nm_tag']:,}")
    
    if ed_stats['mean'] is not None:
        print(f"\n  Summary statistics:")
        print(f"    Mean: {ed_stats['mean']:.2f} mismatches/read")
        print(f"    Median: {ed_stats['median']}")
        print(f"    Min: {ed_stats['min']}")
        print(f"    Max: {ed_stats['max']}")
        
        # Quality assessment
        if ed_stats['mean'] < 2.0:
            print(f"\n  ✅ EXCELLENT: Low mismatch rate (<2.0)")
        elif ed_stats['mean'] < 5.0:
            print(f"\n  ✅ GOOD: Moderate mismatch rate (<5.0)")
        else:
            print(f"\n  ⚠️  WARNING: High mismatch rate (≥5.0)")
        
        # Distribution
        print(f"\n  Edit distance distribution:")
        dist = ed_stats['distribution']
        for nm in sorted(dist.keys())[:10]:  # Top 10
            count = dist[nm]
            pct = 100 * count / ed_stats['with_nm_tag']
            bar = '█' * int(pct / 2)
            print(f"    {nm:2d} mismatches: {bar} {count:,} ({pct:.1f}%)")
    
    # Calculate coverage at positions
    print(f"\n\n📊 COVERAGE ANALYSIS:")
    print(f"{'=' * 70}\n")
    
    reader = biometal.BamReader.from_path(bam_path)
    coverage = defaultdict(int)
    
    # Calculate coverage
    for record in reader:
        if record.reference_id == reference_id and record.is_primary and record.position is not None:
            # Simple coverage counting
            start = record.position
            length = record.reference_length()
            for pos in range(start, start + length):
                coverage[pos] += 1
    
    if coverage:
        cov_values = list(coverage.values())
        print(f"  Reference: {reference_id}")
        print(f"  Positions covered: {len(coverage):,}")
        print(f"  Mean coverage: {statistics.mean(cov_values):.1f}x")
        print(f"  Median coverage: {statistics.median(cov_values):.1f}x")
        print(f"  Min coverage: {min(cov_values)}x")
        print(f"  Max coverage: {max(cov_values)}x")
        
        # Coverage thresholds
        low_cov = sum(1 for c in cov_values if c < 10)
        high_cov = sum(1 for c in cov_values if c >= 30)
        print(f"\n  Coverage thresholds:")
        print(f"    <10x: {low_cov:,} positions ({100*low_cov/len(coverage):.1f}%)")
        print(f"    ≥30x: {high_cov:,} positions ({100*high_cov/len(coverage):.1f}%)")
    
    # NEW v1.4.0: Strand bias at positions
    print(f"\n\n📊 STRAND BIAS ANALYSIS:")
    print(f"{'=' * 70}\n")
    
    # Auto-select positions if not provided (use positions with high coverage)
    if positions is None and coverage:
        # Select 5 positions with highest coverage
        positions = sorted(coverage.items(), key=lambda x: -x[1])[:5]
        positions = [pos for pos, _ in positions]
    
    if positions:
        print(f"  Analyzing {len(positions)} positions (window size: {window_size}bp)\n")
        
        for pos in positions:
            # NEW v1.4.0: Built-in strand bias calculation
            bias = biometal.strand_bias(
                bam_path,
                reference_id=reference_id,
                position=pos,
                window_size=window_size
            )
            
            if bias['total'] > 0:
                print(f"  Position {pos}:")
                print(f"    Total reads: {bias['total']}")
                print(f"    Forward: {bias['forward']} ({100*bias['forward_pct']:.1f}%)")
                print(f"    Reverse: {bias['reverse']} ({100*bias['reverse_pct']:.1f}%)")
                print(f"    Ratio: {bias['ratio']:.2f}:1")
                
                # Assess strand bias
                if 0.4 <= bias['ratio'] <= 2.5:
                    print(f"    ✅ PASS: No significant strand bias")
                else:
                    print(f"    ⚠️  WARN: Potential strand bias artifact")
                print()
    
    print(f"{'=' * 70}")
    print(f"✅ Variant calling QC complete")


# Run variant calling QC
variant_calling_qc(bam_path, reference_id=0, window_size=100)

---

# Workflow 4: RNA-seq Alignment QC

**Goal**: Quality control for RNA-seq alignments (splice-aware).

**Metrics**:
- Alignment length distribution (detects introns)
- Intron-spanning reads (N operations in CIGAR)
- Junction site statistics
- Mapping rate to transcriptome

**Use Case**:
- Validate RNA-seq alignment quality
- Detect splice junctions
- Assess alignment strategy (splice-aware vs naive)

**NEW v1.4.0**: `biometal.alignment_length_distribution()` - Detects intron-spanning reads

In [ ]:
def rnaseq_alignment_qc(bam_path: str, reference_id: int = None):
    """
    RNA-seq alignment quality control using v1.4.0 features.
    
    Args:
        bam_path: Path to BAM file
        reference_id: Optional reference to analyze
    """
    print("🔬 RNA-seq Alignment QC Analysis...\n")
    
    # NEW v1.4.0: Alignment length distribution
    print("📊 ALIGNMENT LENGTH DISTRIBUTION:")
    print(f"={'=' * 70}\n")
    
    length_dist = biometal.alignment_length_distribution(
        bam_path,
        reference_id=reference_id
    )
    
    if not length_dist:
        print("❌ No mapped reads found")
        return
    
    # Calculate statistics
    lengths = []
    for length, count in length_dist.items():
        lengths.extend([length] * count)
    
    total_alignments = len(lengths)
    mean_length = statistics.mean(lengths)
    median_length = statistics.median(lengths)
    
    print(f"  Total alignments: {total_alignments:,}")
    print(f"  Mean alignment length: {mean_length:.1f} bp")
    print(f"  Median alignment length: {median_length} bp")
    print(f"  Min: {min(lengths)} bp")
    print(f"  Max: {max(lengths)} bp")
    
    # Detect intron-spanning reads (long alignments)
    short_alignments = sum(1 for l in lengths if l <= 200)
    medium_alignments = sum(1 for l in lengths if 200 < l <= 1000)
    long_alignments = sum(1 for l in lengths if l > 1000)  # Likely intron-spanning
    
    print(f"\n  Length categories:")
    print(f"    Short (≤200bp): {short_alignments:,} ({100*short_alignments/total_alignments:.1f}%)")
    print(f"    Medium (201-1000bp): {medium_alignments:,} ({100*medium_alignments/total_alignments:.1f}%)")
    print(f"    Long (>1000bp): {long_alignments:,} ({100*long_alignments/total_alignments:.1f}%)")
    
    if long_alignments > 0:
        print(f"\n  ✅ Intron-spanning reads detected (splice-aware alignment)")
    else:
        print(f"\n  ⚠️  No long alignments found (check if splice-aware aligner used)")
    
    # Distribution histogram
    print(f"\n  Top 10 alignment lengths:")
    for length, count in sorted(length_dist.items(), key=lambda x: -x[1])[:10]:
        pct = 100 * count / total_alignments
        bar = '█' * int(pct / 2)
        print(f"    {length:5d}bp: {bar} {count:,} ({pct:.1f}%)")
    
    # Analyze splice junctions (N operations)
    print(f"\n\n📊 SPLICE JUNCTION ANALYSIS:")
    print(f"{'=' * 70}\n")
    
    reader = biometal.BamReader.from_path(bam_path)
    
    junction_stats = {
        'total_reads': 0,
        'with_junctions': 0,
        'junction_count': 0,
        'junction_lengths': [],
    }
    
    for record in reader:
        if reference_id is not None and record.reference_id != reference_id:
            continue
        
        if not record.is_mapped or not record.is_primary:
            continue
        
        junction_stats['total_reads'] += 1
        
        # Check for N operations (introns)
        has_junction = False
        for op in record.cigar:
            if op.op_char == 'N':  # Skipped reference (intron)
                has_junction = True
                junction_stats['junction_count'] += 1
                junction_stats['junction_lengths'].append(op.length)
        
        if has_junction:
            junction_stats['with_junctions'] += 1
    
    print(f"  Total reads analyzed: {junction_stats['total_reads']:,}")
    print(f"  Reads with junctions: {junction_stats['with_junctions']:,} "
          f"({100*junction_stats['with_junctions']/junction_stats['total_reads']:.1f}%)")
    print(f"  Total junctions: {junction_stats['junction_count']:,}")
    
    if junction_stats['junction_lengths']:
        mean_junction = statistics.mean(junction_stats['junction_lengths'])
        median_junction = statistics.median(junction_stats['junction_lengths'])
        
        print(f"\n  Junction length statistics:")
        print(f"    Mean: {mean_junction:.1f} bp")
        print(f"    Median: {median_junction} bp")
        print(f"    Min: {min(junction_stats['junction_lengths'])} bp")
        print(f"    Max: {max(junction_stats['junction_lengths'])} bp")
        
        # Typical intron sizes
        small_introns = sum(1 for l in junction_stats['junction_lengths'] if l < 100)
        medium_introns = sum(1 for l in junction_stats['junction_lengths'] if 100 <= l <= 10000)
        large_introns = sum(1 for l in junction_stats['junction_lengths'] if l > 10000)
        
        print(f"\n  Intron size distribution:")
        print(f"    Small (<100bp): {small_introns:,}")
        print(f"    Medium (100-10kb): {medium_introns:,}")
        print(f"    Large (>10kb): {large_introns:,}")
        
        if medium_introns > small_introns:
            print(f"\n  ✅ Typical intron size distribution detected")
        else:
            print(f"\n  ⚠️  Unusual intron size distribution")
    else:
        print(f"\n  ⚠️  No splice junctions found (check aligner settings)")
    
    print(f"\n{'=' * 70}")
    print(f"✅ RNA-seq QC complete")


# Run RNA-seq QC
rnaseq_alignment_qc(bam_path, reference_id=0)

---

# Workflow 5: Multi-Sample Tag-Based Filtering

**Goal**: Process multi-sample BAM files with read group filtering.

**Metrics**:
- Per-sample read counts
- Per-sample quality metrics
- Sample-specific filtering
- Tag-based extraction

**Use Case**:
- Process multiplexed sequencing data
- Extract per-sample BAM files
- Compare quality across samples
- Demultiplex by read group

**NEW v1.4.0**: Tag convenience methods (`record.read_group()`, `record.get_string()`)

In [ ]:
def multi_sample_analysis(bam_path: str, limit: int = 10000):
    """
    Multi-sample analysis using v1.4.0 tag convenience methods.
    
    Args:
        bam_path: Path to BAM file
        limit: Maximum records to process
    """
    print("🔬 Multi-Sample Analysis (Read Group Based)...\n")
    
    reader = biometal.BamReader.from_path(bam_path)
    
    # Per-sample metrics
    sample_metrics = defaultdict(lambda: {
        'total': 0,
        'mapped': 0,
        'high_quality': 0,
        'edit_distances': [],
        'alignment_scores': [],
    })
    
    total_reads = 0
    reads_without_rg = 0
    
    print("Processing reads...\n")
    
    for record in reader:
        total_reads += 1
        
        # NEW v1.4.0: Read group convenience method
        read_group = record.read_group()
        
        if read_group is None:
            reads_without_rg += 1
            read_group = "NO_RG"  # Default for reads without RG tag
        
        metrics = sample_metrics[read_group]
        metrics['total'] += 1
        
        if record.is_mapped:
            metrics['mapped'] += 1
        
        if record.mapq and record.mapq >= 30:
            metrics['high_quality'] += 1
        
        # NEW v1.4.0: Tag convenience methods
        edit_dist = record.edit_distance()
        if edit_dist is not None:
            metrics['edit_distances'].append(edit_dist)
        
        align_score = record.alignment_score()
        if align_score is not None:
            metrics['alignment_scores'].append(align_score)
        
        if limit and total_reads >= limit:
            break
    
    # Generate per-sample report
    print(f"📊 MULTI-SAMPLE ANALYSIS")
    print(f"={'=' * 70}\n")
    print(f"File: {bam_path}")
    print(f"Total reads analyzed: {total_reads:,}")
    print(f"Samples detected: {len(sample_metrics)}")
    print(f"Reads without RG tag: {reads_without_rg:,}\n")
    
    # Per-sample statistics
    for sample, metrics in sorted(sample_metrics.items()):
        print(f"\nSAMPLE: {sample}")
        print(f"{'-' * 70}")
        
        print(f"  Total reads: {metrics['total']:,} ({100*metrics['total']/total_reads:.1f}% of total)")
        print(f"  Mapped: {metrics['mapped']:,} ({100*metrics['mapped']/metrics['total']:.1f}%)")
        print(f"  High quality (MAPQ≥30): {metrics['high_quality']:,} "
              f"({100*metrics['high_quality']/metrics['total']:.1f}%)")
        
        if metrics['edit_distances']:
            ed_mean = statistics.mean(metrics['edit_distances'])
            ed_median = statistics.median(metrics['edit_distances'])
            print(f"\n  Edit distance (NM):")
            print(f"    Reads with NM: {len(metrics['edit_distances']):,}")
            print(f"    Mean: {ed_mean:.2f}")
            print(f"    Median: {ed_median}")
        
        if metrics['alignment_scores']:
            as_mean = statistics.mean(metrics['alignment_scores'])
            as_median = statistics.median(metrics['alignment_scores'])
            print(f"\n  Alignment score (AS):")
            print(f"    Reads with AS: {len(metrics['alignment_scores']):,}")
            print(f"    Mean: {as_mean:.2f}")
            print(f"    Median: {as_median}")
    
    # Comparative analysis
    print(f"\n\n📊 COMPARATIVE ANALYSIS")
    print(f"={'=' * 70}\n")
    
    # Mapping rate comparison
    print("  Mapping rate by sample:")
    for sample, metrics in sorted(sample_metrics.items(), 
                                  key=lambda x: -x[1]['mapped']/x[1]['total']):
        map_rate = 100 * metrics['mapped'] / metrics['total']
        bar = '█' * int(map_rate / 2)
        print(f"    {sample:15s}: {bar} {map_rate:.1f}%")
    
    # Quality comparison
    print(f"\n  High-quality rate by sample (MAPQ≥30):")
    for sample, metrics in sorted(sample_metrics.items(),
                                  key=lambda x: -x[1]['high_quality']/x[1]['total']):
        hq_rate = 100 * metrics['high_quality'] / metrics['total']
        bar = '█' * int(hq_rate / 2)
        print(f"    {sample:15s}: {bar} {hq_rate:.1f}%")
    
    print(f"\n{'=' * 70}")
    print(f"✅ Multi-sample analysis complete")


# Run multi-sample analysis
multi_sample_analysis(bam_path, limit=10000)

---

# Performance Notes

All workflows leverage biometal's production-grade performance:

### Key Features:
- **4.4 million records/sec** throughput
- **43.0 MiB/s** compressed BAM processing
- **Constant ~5 MB memory** regardless of file size
- **Parallel BGZF decompression** (automatic, 4× speedup)

### Production Tips:
1. **Stream records**: Never accumulate in memory
2. **Filter early**: Use `is_mapped`, `is_primary`, MAPQ thresholds
3. **Built-in functions**: Use v1.4.0 statistics functions (optimized)
4. **Tag access**: Use convenience methods (cached, efficient)

### Memory Usage:
```python
# BAD: Accumulates in memory
records = list(reader)  # 💥 Out of memory on large files

# GOOD: Streaming (constant memory)
for record in reader:   # ✅ Constant ~5 MB
    process(record)
```

---

# Summary

## v1.4.0 Features Demonstrated:

### Tag Convenience Methods:
- ✅ `record.get_int(tag)` - Direct integer access
- ✅ `record.get_string(tag)` - Direct string access
- ✅ `record.edit_distance()` - NM tag (mismatches)
- ✅ `record.alignment_score()` - AS tag
- ✅ `record.read_group()` - RG tag
- ✅ `record.md_string()` - MD tag

### Statistics Functions:
- ✅ `biometal.insert_size_distribution()` - Paired-end QC
- ✅ `biometal.edit_distance_stats()` - Alignment quality
- ✅ `biometal.strand_bias()` - Variant calling QC
- ✅ `biometal.alignment_length_distribution()` - RNA-seq QC

## Production Workflows:

1. **Quality Control Pipeline** - Comprehensive metrics (mapping, quality, tags)
2. **Paired-End Analysis** - Insert size distribution and QC
3. **Variant Calling Prep** - Coverage, strand bias, edit distance
4. **RNA-seq QC** - Splice junctions and intron detection
5. **Multi-Sample Processing** - Read group filtering and comparison

## Performance:
- **4.4 million records/sec** throughput
- **Constant ~5 MB memory** (scales to TB files)
- **Automatic optimization** (parallel BGZF, no config needed)

---

## Next Steps

### For Your Data:
1. Replace `bam_path` with your alignment files
2. Adjust filtering thresholds (MAPQ, coverage, etc.)
3. Integrate into production pipelines
4. Scale to large cohorts (TB-scale works!)

### Additional Resources:
- **API Reference**: `docs/BAM_API.md`
- **Performance Guide**: `docs/BAM_PERFORMANCE.md`
- **Basic Tutorial**: `notebooks/05_bam_alignment_analysis.ipynb`
- **GitHub**: https://github.com/shandley/biometal
- **Issues**: https://github.com/shandley/biometal/issues

---

**biometal v1.4.0** - Production-ready BAM processing